In [ ]:
# suppress qiskit 1.0 deprecation warnings:
import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

from qiskit import QuantumCircuit, Aer, execute
import src.qcm_sim as sim
import src.compile_keys as keys

# import circuits:
import circuits.dj_circuit as dj_circuit

# Deutsch-Jozsa Algorithm

## Background

Deutsch-Jozsa problem [[Deutsch, Jozsa, 1992](https://royalsocietypublishing.org/doi/abs/10.1098/rspa.1992.0167)] is to decide if a function $f : \mathbb{F}_2^n \to \mathbb{F}_2 $ is **constant** or **balanced** with as few queries to an oracle as possible. Classically one requires $2^{n-1}+1$ queries to decide this deterministically in the worst case. On the other hand, the Deutsch-Jozsa algorithm gives a gives a quantum solution requiring just a single query to a quantum oracle.

A function $f : \mathbb{F}_2^n \to \mathbb{F}_2 $ is called:
​
- **constant** if $ f(x) = b$, for some $b\in \mathbb{F}_2$ and for all $x\in \mathbb{F}_2^n$,

- **balanced** if $ |f^{-1}(0)| = |f^{-1}(1)| $.
​

## Quantum solution

We are given access to an oracle $O_{f}$ which is an $(n+1)$-qubit unitary operator whose action on the computational basis is given by $O_{f}\lvert y,z\rangle = \lvert y, z + f(y)\rangle$ with $y\in\mathbb{F}_{2}^{n}$ and where the sum is taken modulo $2$. Implementing the unitary $U = H^{\otimes n}O_{f}H^{\otimes (n+1)}$ we have a final quantum state given by $\lvert \psi\rangle = U\lvert 0_n, 1\rangle $ wherein $\langle  \psi\rvert (\Pi^{0_n}\otimes I)\lvert  \psi\rangle =0,1$ if $f$ is balanced, constant, respectively.  See e.g., [[Nielsen, Chuang, 2010](https://www.cambridge.org/highereducation/books/quantum-computation-and-quantum-information/01E10196D0A682A6AEFFEA52D53BE9AE#overview)] for additional details.

# Code for Deutsch-Jozsa Circuit:

## Constant oracle

In [ ]:
def create_constant_oracle(n_qubits, output):
    """
    Creates a 'constant' oracle.

    If `output` is 0, the oracle always returns 0.
    If `output` is 1, the oracle always returns 1.

    Args:
        n_qubits (int): The number of input qubits.
        output (int): The constant output value of the function (0 or 1).

    Returns:
        QuantumCircuit: A quantum circuit implementing the constant oracle.
    """
    oracle = QuantumCircuit(n_qubits)

    # If the oracle should always output 1, we flip the "output" qubit
    # using an X-gate (think of it as a NOT gate on a qubit).
    if output == 1:
        oracle.x(n_qubits)

    return oracle

## Balanced oracle

There is many ways to create a balanced function. For example, the function $f(w,x,y) = wx+y$ creates a balanced function  
| input | output | 
|----------|----------|
| 0 0 0   | 0   |
| 1 0 1 | 1   | 
| 0 1 0   | 0   |
| 0 0 1   | 0   |
| 1 1 0 | 1   | 
| 0 1 1   | 1   |
| 1 0 1   | 1   |
| 1 1 1 | 0   |


We can write the output of this function onto an output register using a Toffoli gate and CNOT gate. To see this, recall that the Toffoli gate shifts the output register by a quadratic function

$$(x,y,z)\mapsto (x,y,z\oplus xy)$$

and the CNOT gate shifts the output by a linear offset $(x,y)\mapsto (x,y\oplus x)$. The function $f$ can then be written to an output register using a Toffoli and CNOT gate in sequence

$$(w,x,y,z) \mapsto (w,x,y,z\oplus wx) \mapsto (w,x,y,z\oplus wx \oplus y) = (w,x,y,z\oplus f(w,x,y)).$$

**Lemma**: Given two balanced functions for the different registers $f_1(x_1)$ and $f_2(x_2)$, the function $g(x_1,x_2)=f_1(x_1)\oplus f_2(x_2)$ will be balanced.

Our strategy for constructing balanced quantum oracles will be based on applying the balanced function $f$ on disjoint subsets of $\mathbb{F}_2^n$, which by the above Lemma gives another balanced function.

## Construction of quantum oracle 

Our explicit construction of the oracle $O_{f}$ is as follows. Fix some positive integer $c$ and let $n\geq 3c$. We consider Boolean functions $f:\mathbb{F}_{2}^{n}\to \mathbb{F}_{2}$ whose polynomial degree is at most quadratic in the inputs. When $f$ is constant we simply take $f(y)=0$ for all $y\in \mathbb{F}_{2}^{n}$ so that $O_{f}$ is just the identity. For the balanced case we take
$$
f(y) = \sum_{j=0}^{c-1}y_{j_{1}}y_{j_{2}}+y_{j_{3}}+\sum_{k=3c+1}^{n}y_{k}.
$$
The function $f$ is readily seen to be a balanced function since it is a sum of balanced functions with disjoint domains in $\mathbb{F}_{2}^{n}$. To implement the quantum oracle recall first that the action of $CX$ and the Toffoli gate $CCX$ on the computational basis is $CX_{c,t}\lvert  y_{c},y_{t}\rangle =\lvert  y_{c},y_{t}+y_{c} \rangle $ and $CCX_{c_{1},c_{2},t}\lvert y_{c_{1}},y_{c_{2}},y_{t}\rangle =\lvert  y_{c_{1}},y_{c_{2}},y_{t}+y_{c_{1}}y_{c_{2}}\rangle $, respectively. We then have that the unitary implementing the oracle $O_{f}$ is given by
$$
O_{f}= \prod_{j=0}^{c-1} CCX_{j_{1},j_{2},j_{3}}\,CX_{j_{3},t} ~ \prod_{k=3c+1}^{n} CX_{k,t}.
$$
To our knowledge the Deutsch-Jozsa algorithm has not previously been used to benchmark Clifford+$T$ circuits. One advantage of our Deutsch-Jozsa oracle over that used in the hidden shift algorithm is that ours is more flexible in its use of non-Clifford gates since it can allow for any number of $CCX$ gates while the hidden shift oracle always requires an even number of $CCZ = (I\otimes I\otimes H)\, CCX \, (I\otimes I\otimes H)$ gates.

In [ ]:
def create_balanced_oracle(toff_count, n_qubits):
    """
    Creates a 'balanced' oracle.

    Half of the input bit patterns output 0, and the other half output 1.

    Args:
        toff_count (int): The number of Toffoli gates in the desired oracle.
        n_qubits (int): The number of qubits int the oracle.

    Returns:
        QuantumCircuit: A quantum circuit implementing the balanced oracle.
    """
    
    oracle=QuantumCircuit(n_qubits)
    toff_part=3*toff_count+1
    if n_qubits>=toff_part:
        for i in range(0,toff_part-1,3):
            oracle.ccx(i, i+1, i+2)
            oracle.cx(i+2, n_qubits-1)
        for i in range(toff_part-1,n_qubits-1,1):
            oracle.cx(i,n_qubits-1)

        return oracle
    else:
        print("The total qubit count must be greater or equal to three times the number of Toffoli gates plus one.")

## Born rule estimation: constant oracle

In [ ]:
n = 10
toff_count = 1      # doesn't matter for constant case

# Test deutsch jozsa on constant:
qc = dj_circuit.run_deutsch_jozsa_test(n, toff_count,'constant',0)
qc_qasm = qc.qasm()

In [ ]:
# number of samples:
number_of_t_gates = 7*toff_count
epsilon = 0.1
prob_fail = 0.1
hoeffding_samples = keys.compute_hoeffding_samples(number_of_t_gates,epsilon,prob_fail)

In [ ]:
# QCM simulation
constant_results = sim.run_qcm(qc_qasm,shots = hoeffding_samples)

## Born rule estimation: balanced oracle

In [ ]:
n = 10
toff_count = 1

# Test deutsch jozsa on constant:
qc = dj_circuit.run_deutsch_jozsa_test(n, toff_count,'balanced',0)
qc_qasm = qc.qasm()

In [ ]:
# number of samples:
number_of_t_gates = 7*toff_count
epsilon = 0.1
prob_fail = 0.1
hoeffding_samples = keys.compute_hoeffding_samples(number_of_t_gates,epsilon,prob_fail)

In [ ]:
# QCM simulation
balanced_results = sim.run_qcm(qc_qasm,shots = hoeffding_samples)